In [8]:
import sklearn.ensemble as sle
import sklearn.linear_model as sllm
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np
from sklearn import metrics
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
import csv
import numpy as np

In [9]:
# Binary classification: 3 vs. 8
mnist_mini = scipy.io.loadmat('MNISTmini.mat')
X_raw = mnist_mini['train_fea1'] / 255
Y_raw = mnist_mini['train_gnd1'].T.flatten()
X_test = mnist_mini['test_fea1']
Y_test = mnist_mini['test_gnd1'].T.flatten()

mask_train = np.logical_or(Y_raw == 3, Y_raw == 8)
mask_test = np.logical_or(Y_test == 3, Y_test == 8)

X_raw_mask = X_raw[mask_train]
Y_raw_mask = Y_raw[mask_train]

X_test = X_test[mask_test]
Y_test = Y_test[mask_test]

In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(X_raw_mask,Y_raw_mask,test_size=0.3)

In [11]:
print(X_train.shape,X_val.shape)

(8387, 100) (3595, 100)


In [12]:
forest_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True,False]}

log_model = sllm.LogisticRegression(penalty='l2',solver='liblinear',tol=0.01).fit(X_train,Y_train)
random_forest = sle.RandomForestClassifier()
random_forest_search = RandomizedSearchCV(estimator = random_forest, param_distributions = forest_grid, n_iter = 10, cv = 3, verbose=2, n_jobs = -1)
random_forest_search.fit(X_train,Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=2)

In [13]:
Y_log_pred = log_model.predict(X_test)
print(metrics.confusion_matrix(Y_test,Y_log_pred))
print(log_model.get_params())

[[923  87]
 [ 10 964]]
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 0.01, 'verbose': 0, 'warm_start': False}


In [14]:

Y_forest_pred_search = random_forest_search.predict(X_test)
print(metrics.classification_report(Y_test, Y_forest_pred_search))
print(metrics.confusion_matrix(Y_test, Y_forest_pred_search))
print(metrics.r2_score(Y_test, Y_forest_pred_search))
print(metrics.mean_squared_log_error(Y_test, Y_forest_pred_search))
print(random_forest_search.best_params_)
print(random_forest_search.best_score_)

              precision    recall  f1-score   support

           3       0.99      0.98      0.98      1010
           8       0.98      0.99      0.98       974

    accuracy                           0.98      1984
   macro avg       0.98      0.98      0.98      1984
weighted avg       0.98      0.98      0.98      1984

[[990  20]
 [ 14 960]]
0.931429036127432
0.01126
